In [2]:
import pandas as pd
import numpy as np
from prophet import Prophet  # pip install prophet
from datetime import timedelta

# ─── 1. Load & prepare data ────────────────────────────────────────────────────
# Your two CSVs from the previous step
dl = pd.read_csv('delivery_log.csv', parse_dates=['date_time'])
cr = pd.read_csv('consumption_record.csv', parse_dates=['date'])

# Compute daily shipments (outflow) per warehouse
ship = (
    dl.assign(date=dl['date_time'].dt.floor('D'))
      .groupby(['warehouse_id','date'])
      .quantity_shipped.sum()
      .reset_index()
      .rename(columns={'quantity_shipped':'outflow'})
)

# (If you had inflow/returns, subtract here: net = inflow - outflow)

# ─── 2. Forecast with Prophet ─────────────────────────────────────────────────
def fit_and_forecast(df, periods=30):
    # Prophet expects columns ['ds','y']
    m = Prophet(
        yearly_seasonality=False,
        weekly_seasonality=True,
        daily_seasonality=False
    )
    m.fit(df)  # :contentReference[oaicite:0]{index=0}
    future = m.make_future_dataframe(periods=periods, freq='D')
    fcst = m.predict(future)
    return fcst[['ds','yhat']]

# Prepare a place to collect forecasts
forecasts = {}
for w in ship.warehouse_id.unique():
    hist = ship[ship.warehouse_id==w][['date','outflow']].rename(
        columns={'date':'ds','outflow':'y'}
    )
    fc = fit_and_forecast(hist)
    forecasts[w] = fc

# ─── 3. Compute current stock & days‑to‑depletion ─────────────────────────────
# (In a real system you'd pull this from your WMS. Here we mock it.)
initial_stock = {'W1': 5000, 'W2': 6000}  

alerts = []
lead_time_days = 7  # example lead time

for w, fc in forecasts.items():
    # current stock level
    S = initial_stock.get(w, 0)
    
    # cumulative forecasted demand
    fc = fc.copy()
    fc['cum_demand'] = fc['yhat'].clip(lower=0).cumsum()
    
    # 4. Reorder‑Point (ROP) & Safety Stock
    hist = ship[ship.warehouse_id==w]
    avg_daily = hist.outflow.mean()
    max_daily = hist.outflow.max()
    # safety stock = (max_daily*LT) – (avg_daily*LT)
    safety_stock = (max_daily * lead_time_days) - (avg_daily * lead_time_days)
    # ROP = avg_daily*LT + safety_stock  :contentReference[oaicite:1]{index=1}
    reorder_point = avg_daily * lead_time_days + safety_stock
    
    # Find first day where cum_demand >= (S – ROP)
    threshold = S - reorder_point
    soon = fc[fc.cum_demand >= threshold]
    if not soon.empty:
        days_to_depletion = (soon.iloc[0].ds - pd.Timestamp.today().normalize()).days
    else:
        days_to_depletion = np.nan
    
    alerts.append({
        'warehouse_id': w,
        'current_stock': S,
        'avg_daily_demand': avg_daily,
        'safety_stock': safety_stock,
        'reorder_point': reorder_point,
        'days_to_depletion': days_to_depletion
    })

alert_df = pd.DataFrame(alerts)
print(alert_df)

# ─── 5. Trigger restock if within lead time ───────────────────────────────────
to_reorder = alert_df[
    alert_df.days_to_depletion <= lead_time_days
].warehouse_id.tolist()

if to_reorder:
    print(f"🔔 Warehouses to restock within {lead_time_days} days:", to_reorder)
else:
    print("All warehouses have sufficient stock for the next", lead_time_days, "days.")


C:\Users\T3LQGUT\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Matplotlib is building the font cache; this may take a moment.
Importing plotly failed. Interactive plots will not work.
10:05:40 - cmdstanpy - INFO - Chain [1] start processing
10:05:42 - cmdstanpy - INFO - Chain [1] done processing
10:05:42 - cmdstanpy - INFO - Chain [1] start processing
10:05:42 - cmdstanpy - INFO - Chain [1] done processing


  warehouse_id  current_stock  avg_daily_demand  safety_stock  reorder_point  \
0           W1           5000         341.87500    3535.87500         5929.0   
1           W2           6000         450.21875    3148.46875         6300.0   

   days_to_depletion  
0                -30  
1                -30  
🔔 Warehouses to restock within 7 days: ['W1', 'W2']
